# 导包

In [1]:
# 将代码包路径导入python搜索路径, 导入一些必要模块
import os
import copy
import sys
# 该部分为python搜索路径，调用本地的代码包，若使用服务器环境，需注释掉该部分内容
package_path_2 = r'/Users/liliangshan/workspace/python/daily_code/tl_modeler'
sys.path.insert(0, package_path_2)

import pandas as pd
import matplotlib.pyplot as plt

import tl_modeler as tlm

In [2]:
# path_project
path_project = r'/Users/liliangshan/workspace/python/01_datasets/AMEX_Default_Prediction'

# 文件夹
path_row_data = os.path.join(path_project, 'row_data')
path_new_data = os.path.join(path_project, 'new_data')
path_results = os.path.join(path_project, 'results')

# row_data
path_row_train = os.path.join(path_row_data,  'train_data.ftr')
path_row_test  = os.path.join(path_row_data, 'test_data.ftr')
path_train_label = os.path.join(path_row_data, 'train_labels.csv')



## results
path_output_report = os.path.join(path_results, 'scorecard_report.xlsx')

y_label = 'target'

# 数据读取

In [3]:
df_labels = tlm.read_data(path_train_label)
df_train = tlm.read_data(path_row_train).drop(columns=[y_label])

features = df_train.columns.drop(['customer_ID', 'S_2']).to_list()

In [4]:
cat_features = [
    "B_30",
    "B_38",
    "D_114",
    "D_116",
    "D_117",
    "D_120",
    "D_126",
    "D_63",
    "D_64",
    "D_66",
    "D_68",
]

num_features = [col for col in features if col not in cat_features]

df_train[cat_features] = df_train[cat_features].astype('object')

In [5]:
train_num_agg = df_train.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last'])
train_num_agg.columns = ['_'.join(x) for x in train_num_agg.columns]
train_num_agg.reset_index(inplace = True)

train_cat_agg = df_train.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])
train_cat_agg.columns = ['_'.join(x) for x in train_cat_agg.columns]
train_cat_agg.reset_index(inplace = True)

df_train = pd.merge(df_labels, train_num_agg, how='left', on = 'customer_ID')
df_train = pd.merge(df_train, train_cat_agg, how='left', on = 'customer_ID')

## 数据准备

In [6]:
# df_train = df_train.groupby("customer_ID").tail(1).reset_index(drop=True)
# #df_test = df_test.groupby("customer_ID").tail(1).reset_index(drop=True)

In [7]:
df_train.head()

,customer_ID,target,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,...,D_63_nunique,D_64_count,D_64_last,D_64_nunique,D_66_count,D_66_last,D_66_nunique,D_68_count,D_68_last,D_68_nunique
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0,0.933594,0.024194,0.868652,0.960449,0.934570,0.010704,0.024440,0.001082,...,1,13,O,1,0,None,0,13,6.0,1
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0,0.899902,0.022097,0.861328,0.929199,0.880371,0.215210,0.199123,0.002224,...,1,13,O,1,0,None,0,13,6.0,1
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0,0.878418,0.028837,0.797852,0.904297,0.880859,0.004181,0.002759,0.000802,...,1,13,R,1,0,None,0,13,6.0,1
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0,0.599121,0.020082,0.567383,0.623535,0.621582,0.048859,0.088490,0.000660,...,1,13,O,1,0,None,0,13,3.0,3
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0,0.891602,0.042316,0.805176,0.940430,0.872070,0.004642,0.002883,0.000030,...,1,13,O,1,13,1.0,1,13,6.0,1


## 数据预处理

In [8]:
trans_train, filter_cols, missing_cols, single_list, mutitype_list, varaince_list = tlm.feature_filter(df_train, y_label)

In [9]:
missing_threshold=0.9
single_threshold=0.9  
variance_threshold=0
unique_threshold=40    
feat_ = '字段名'

analysis = pd.DataFrame(data={'字段名': df_train.columns.tolist()})

analysis['缺失率过滤'] = analysis[feat_].map(lambda x: '02.缺失率>={},被移除'.format(missing_threshold) if x in missing_cols else '')
analysis['单一值、百分比过高特征过滤'] = analysis[feat_].map(lambda x: '03.单一值、百分比>={},被移除'.format(single_threshold) if x in single_list else '')
analysis['类别过多特征过滤'] = analysis[feat_].map(lambda x: '04.类别取值个数>={},被移除'.format(unique_threshold) if x in mutitype_list else '')
analysis['方差过小过滤'] = analysis[feat_].map(lambda x: '05.方差<={},被移除'.format(variance_threshold) if x in varaince_list else '')

title = "数据预过滤"
tlm.dat2excel(analysis, title, sheet_name='06.数据预处理', excel_path=path_output_report)

## 分箱及分箱调整

In [10]:
bins_adj = tlm.woebin(trans_train, y=y_label)

There are blank strings in 1 columns, which are replaced with NaN. 
 (ColumnNames: D_64_last)
There are 1 variables have only one binning intervals，please check the binning result. 
 (ColumnNames: D_66_last)


In [11]:
# 调单调变量
bins, res_monotone_list, res_un_monotone_list= tlm.woebin_monotone_adj(trans_train, y=y_label, 
                                                                bins=bins_adj, cols=list(bins_adj.keys()))

特征D_66_last，除特殊值分箱外，分箱箱数少于2箱，无法调整, 建议删除
特征S_18_last, 单调分箱调整失败
特征D_109_last, 单调分箱调整失败


In [12]:
bins = {key: item for key, item in bins.items() if key in res_monotone_list}

In [13]:
train_woe = tlm.woebin_ply(trans_train[res_monotone_list + [y_label]], bins)

[INFO] converting into woe values ...


There are blank strings in 1 columns, which are replaced with NaN. 
 (ColumnNames: D_64_last)


Woe transformating on 458913 rows and 831 columns in 00:11:57


In [14]:
result = pd.DataFrame(data={'字段名': trans_train.columns.tolist()})
result['单调变量调整成功'] = result['字段名'].map(lambda x: '单调变量调整成功' if x in res_monotone_list else '')
result['单调变量调整失败'] = result['字段名'].map(lambda x: '单调变量调整失败' if x in res_un_monotone_list else '')

title = '调整后分箱结果'
tlm.dict2excel(bins, title=title, sheet_name='09.调整后分箱结果', excel_path=path_output_report)

In [15]:
train_woe.head()

,target,S_20_std_woe,R_27_last_woe,B_15_mean_woe,D_119_last_woe,R_13_last_woe,S_16_std_woe,D_125_last_woe,S_9_last_woe,D_143_mean_woe,...,R_20_mean_woe,D_121_max_woe,B_32_min_woe,D_50_std_woe,D_104_mean_woe,R_10_last_woe,S_20_mean_woe,R_2_min_woe,D_70_last_woe,S_9_max_woe
0,0,0.115,-0.285,-0.110,0.019,-0.078,-0.128,-0.056,0.036,-0.145,...,-0.167,-0.375,0.037,-0.166,0.299,-0.204,0.472,0.048,-0.518,0.045
1,0,0.115,-0.285,0.013,-0.357,-0.078,-0.128,-0.056,0.506,-0.145,...,-0.167,0.216,-0.063,0.273,-0.277,-0.204,-0.096,-0.122,-0.518,0.045
2,0,-0.087,-0.285,0.024,-0.357,-0.078,-0.128,-0.056,-0.218,-0.145,...,-0.167,0.216,-0.063,0.273,-0.277,-0.204,-0.096,0.048,-0.518,-0.218
3,0,-0.087,-0.285,0.024,0.378,-0.078,-0.128,-0.056,0.036,-0.145,...,-0.167,0.216,0.037,-0.847,-0.277,-0.204,-0.096,0.048,-0.518,0.045
4,0,0.115,-0.285,0.013,-0.357,-0.102,-0.128,-0.056,-0.218,-0.145,...,-0.167,0.216,0.323,-0.394,0.299,-0.204,-0.096,-0.122,-0.518,-0.218


## 特征筛选

In [16]:
iv_list, iv_drop_var, train_woe = tlm.feature_selection_iv(train_woe, y_label)
chi_drop_var, train_woe = tlm.feature_selection_chi(train_woe, bins=bins, y=y_label)
corr_list, corr_drop_var, train_woe = tlm.feature_selection_corr(train_woe, y_label)
vif_list, vif_drop_var, train_woe = tlm.feature_selection_vif(train_woe, y_label)

变量B_15_mean 的分箱结果未通过卡方检验
变量S_16_std 的分箱结果未通过卡方检验
变量R_20_std 的分箱结果未通过卡方检验
变量D_107_std 的分箱结果未通过卡方检验
变量D_81_std 的分箱结果未通过卡方检验
变量D_79_std 的分箱结果未通过卡方检验
变量R_2_std 的分箱结果未通过卡方检验
变量D_82_mean 的分箱结果未通过卡方检验
变量D_65_std 的分箱结果未通过卡方检验
变量S_17_min 的分箱结果未通过卡方检验
变量D_133_std 的分箱结果未通过卡方检验
变量R_5_std 的分箱结果未通过卡方检验
变量S_25_std 的分箱结果未通过卡方检验
变量R_8_last 的分箱结果未通过卡方检验
变量D_141_last 的分箱结果未通过卡方检验
变量R_24_std 的分箱结果未通过卡方检验
变量S_20_last 的分箱结果未通过卡方检验
变量D_129_std 的分箱结果未通过卡方检验
变量D_84_last 的分箱结果未通过卡方检验
变量R_11_last 的分箱结果未通过卡方检验
变量R_10_std 的分箱结果未通过卡方检验
变量D_130_last 的分箱结果未通过卡方检验
变量D_127_last 的分箱结果未通过卡方检验
变量B_21_std 的分箱结果未通过卡方检验
变量D_72_std 的分箱结果未通过卡方检验
变量R_4_std 的分箱结果未通过卡方检验
变量D_69_last 的分箱结果未通过卡方检验
变量R_25_last 的分箱结果未通过卡方检验
变量R_7_last 的分箱结果未通过卡方检验
变量B_41_last 的分箱结果未通过卡方检验
变量R_8_std 的分箱结果未通过卡方检验
变量B_21_last 的分箱结果未通过卡方检验
变量R_24_last 的分箱结果未通过卡方检验
变量R_7_std 的分箱结果未通过卡方检验
变量D_123_std 的分箱结果未通过卡方检验
变量R_23_last 的分箱结果未通过卡方检验
变量D_123_last 的分箱结果未通过卡方检验
变量R_15_std 的分箱结果未通过卡方检验
变量B_32_std 的分箱结果未通过卡方检验
变量S_6_last 的分箱结果未通过卡方检验
变量D_84_std 的分箱结果未通过卡方检

In [ ]:
stepwise_result, stepwise_drop_list, train_woe = tlm.stepwise(train_woe, y_label, remove_negitive_coef=False)

变量P_2_last_woe被加入
变量B_18_last_woe被加入
变量S_3_mean_woe被加入
变量R_1_last_woe被加入
变量B_9_last_woe被加入
变量D_39_last_woe被加入
变量R_3_mean_woe被加入
变量D_44_last_woe被加入
变量D_56_max_woe被加入
变量B_23_std_woe被加入
变量D_41_last_woe被加入
变量D_132_mean_woe被加入
变量P_3_last_woe被加入
变量D_42_min_woe被加入
变量B_22_mean_woe被加入
变量R_2_last_woe被加入
变量D_50_std_woe被加入
变量B_4_max_woe被加入
变量S_6_mean_woe被加入
变量D_46_mean_woe被加入
变量S_23_last_woe被加入
变量D_43_last_woe被加入
变量D_51_last_woe被加入
变量D_54_min_woe被加入
变量B_21_mean_woe被加入
变量S_26_max_woe被加入
变量D_60_max_woe被加入
变量S_15_max_woe被加入
变量S_22_last_woe被加入
变量B_22_min_woe被加入
变量D_65_last_woe被加入
变量R_26_mean_woe被加入
变量D_71_last_woe被加入
变量D_50_last_woe被加入
变量S_12_std_woe被加入
变量B_5_mean_woe被加入
变量S_9_std_woe被加入
变量B_25_mean_woe被加入
变量D_39_max_woe被加入
变量D_66_count_woe被加入
变量D_47_min_woe被加入
变量S_11_mean_woe被加入
变量B_25_std_woe被加入
变量P_4_std_woe被加入
变量B_17_min_woe被加入
变量R_1_mean_woe被加入
变量D_46_last_woe被加入
变量D_45_max_woe被加入
变量D_82_min_woe被加入
变量D_123_mean_woe被加入
变量D_60_mean_woe被加入
变量D_121_max_woe被加入
变量D_144_mean_woe被加入
变量D_76_last_woe被加入
变量

In [ ]:
coef_result, coef_drop_list, train_woe = tlm.feature_selection_coef(train_woe, y_label)

In [ ]:
iv_min_threshold = 0.02
iv_max_threshold = 0.8
corr_threshold = 0.7
vif_threshold = 10
chi_threshold = 0.05

analysis = pd.DataFrame(data={'字段名': train_woe.columns.tolist()})

analysis['IV 筛选'] = analysis['字段名'].map(lambda x: '06.IV值<{}或IV值>{},被移除'.format(iv_min_threshold, iv_max_threshold) if x in iv_drop_var else '')
analysis['卡方检验'] = analysis['字段名'].map(lambda x: '08.卡方未大于阈值,被移除' if x in chi_drop_var else '')
analysis['相关系数筛选'] = analysis['字段名'].map(lambda x: '09.相关系数>{},被移除'.format(corr_threshold) if x in corr_drop_var else '')
analysis['VIF 筛选'] = analysis['字段名'].map(lambda x: '10.VIF值>{},被移除'.format(vif_threshold) if x in vif_drop_var else '')

analysis['逐步回归筛选'] = analysis['字段名'].map(lambda x: '11.逐步回归筛选,被移除' if x in stepwise_drop_list else '')
analysis['剔除系数为负'] = analysis['字段名'].map(lambda x: '12.剔除系数为负' if x in coef_drop_list else '')

tlm.dat2excel(analysis, '特征筛选', sheet_name='10.特征筛选', excel_path=path_output_report)

## 模型构建

In [ ]:
model = tlm.get_lr_model(train_woe, y_label)

## 评分卡

In [ ]:
card = tlm.scorecard(bins, model, train_woe.columns.drop(y_label))

## 评分卡映射

In [ ]:
res_train = tlm.scorecard_ply(df_train, card)

In [ ]:
res_train['prob'] = model.predict_proba(train_woe.drop(columns=y_label))[:,1]

In [ ]:
res_train[y_label] = df_train[y_label]

In [ ]:
res_train.shape

## 模型评估

In [ ]:
res = tlm.report_model({'train': res_train, 'test': copy.deepcopy(res_train)}, target=y_label, pred='score', 
                       plot··_type=["ks",  "roc", "score_distr"], direction='negative')

## 比赛指标

In [ ]:
def amex_metric(y_true, y_pred, return_components=False) -> float:
    """Amex metric for ndarrays"""
    def top_four_percent_captured(df) -> float:
        """Corresponds to the recall for a threshold of 4 %"""
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(df) -> float:
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(df) -> float:
        """Corresponds to 2 * AUC - 1"""
        df2 = pd.DataFrame({'target': df.target, 'prediction': df.target})
        df2.sort_values('prediction', ascending=False, inplace=True)
        return weighted_gini(df) / weighted_gini(df2)

    df = pd.DataFrame({'target': y_true.ravel(), 'prediction': y_pred.ravel()})
    df.sort_values('prediction', ascending=False, inplace=True)
    g = normalized_weighted_gini(df)
    d = top_four_percent_captured(df)

    if return_components: return g, d, 0.5 * (g + d)
    return 0.5 * (g + d)

In [ ]:
amex_metric(res_train[y_label], res_train['prob'])